<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projetos de Data Science
</h1>

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este Notebook concentra a descrição do Projeto de Análise de Churn para a empresa Telco.  </i> 
</p>  

# <font color='red' style='font-size: 40px;'> Consultas </font>
<hr style='border: 2px solid red;'>

> CONSULTAR:

https://www.kaggle.com/datasets/datacertlaboratoria/projeto-5

https://www.kaggle.com/code/datacertlaboratoria/guia-de-resolu-o-projeto-5

# <font color='red' style='font-size: 40px;'> Problemática </font>
<hr style='border: 2px solid red;'>

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> TELCO é uma empresa jovem e com muito potencial, cujas filiais estão localizadas em 1.106 cidades do estado da Califórnia, Estados Unidos.  </i> 
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> A CEO da empresa queria entender como diminuir a perda de clientes na TELCO, portanto, durante as apresentações, um dos Diretores disse as seguintes hipóteses: 

    1. A perda de clientes na TELCO se deve ao aumento da concorrência. Há cada vez mais concorrentes no mercado, principalmente na região de San Diego, o que nos faz perder mais clientes lá do que em outros lugares.

    2. O problema são os contratos mensais. Se pudéssemos migrar todos os nossos clientes para contratos mais longos, todos os nossos problemas seriam resolvidos.

 </i> 
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Após ouvir as declarações do Diretor, a CEO disse: “É claro para mim que os contratos mês a mês são mais propensos à perda do que os outros, mas como sabemos se esse efeito é o mesmo para todas as idades, ou se é o mesmo para os clientes com mais ou menos antiguidade? Da mesma forma, um cliente que indicou muitos outros clientes tem menos probabilidade de sair, mas como sabemos se o efeito mês a mês é mais ou menos importante do que o número de referências de um cliente? Além disso, assumindo que a estratégia que você propõe acaba sendo a correta, ou seja, em geral buscamos movimentar os clientes para contratos mais longos, como priorizamos? Quem contatamos primeiro? Gostaria de fazer uma análise profunda da situação para entendê-la melhor.”
 </i> 
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Dado esta problemática, ficou acordado o desenvolvimento de um relatório analítico com o seguinte:

    1. Uma visão geral da população de clientes TELCO. A CEO quer entender as características dos clientes TELCO. Qual é a divisão por sexo? Eles têm dependentes? idades? cidades? etc.

    2. Uma exploração de perdas para entender melhor as suposições do Diretor sobre contratos mensais e a região de San Diego como a mais afetada. Algumas segmentações para entender a perda de clientes que saíram durante a reunião são:

    - Grupo 1: Contrato mes-a-mes e idade > 64
    - Grupo 2: Contrato mes-a-mes , idade < 64 e número de referências < 2
    - Grupo 3: Contrato diferente de mes-a-mes, edad > 64 y número de referidos < 2
    - Grupo 4: Contrato diferente a mes-a-mes e antiguidade em meses < 40

    3. Uma análise de clientes em “alto risco” de abandono priorizada de acordo com o valor monetário estimado que representam para a TELCO. Aqui a ideia é que após identificar os fatores que estão relacionados a perda de clientes (com base nos clientes que já saíram), você possa identificar quais clientes, da base de clientes que a TELCO ainda tem (ou seja, clientes que não saíram), se eles seriam mais propensos a deixar a empresa. E a partir dessa base, identifique aqueles clientes que possuem um alto valor monetário para a TELCO. Em outras palavras, você deseja priorizar aqueles clientes que são: i) propensos a sair (não saíram, mas estariam em risco de sair) e ii) de alto valor monetário. Amalia quer a lista desses clientes.
 </i> 
</p>  


# <font color='red' style='font-size: 40px;'> Bibliotecas Utilizadas </font>
<hr style='border: 2px solid red;'>

In [1]:
## Bibliotecas De Manipulação de Dados e Visualização

import pandas as pd 
from pandasql import sqldf
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px 
import plotly.graph_objects as go

## Bibliotecas de Modelagem Matemática e Estatística
import numpy as np
import scipy as sp 
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Bibliotecas de Manipulação de Tempo
import time
import datetime


# Bibliotecas de Suporte
import warnings
import pyodbc



# Parâmetros de Otimização
%matplotlib inline
sns.set()
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# <font color='red' style='font-size: 40px;'> Conexão SQL Server </font>
<hr style='border: 2px solid red;'>

In [2]:
dados_conexao = (
    'Driver={SQL Server};'
    'Server=DESKTOP-A7BL36E;'
    'Database=TELCO;'
)

conexao = pyodbc.connect(dados_conexao)
print('Conexao Bem Sucedida')

cursor = conexao.cursor()

Conexao Bem Sucedida


In [3]:
demographics = pd.read_sql('SELECT * FROM demographics', conexao)
locations = pd.read_sql('SELECT * FROM locations', conexao)
populations = pd.read_sql('SELECT * FROM populations', conexao)
services = pd.read_sql('SELECT * FROM services', conexao)
status = pd.read_sql('SELECT * FROM status', conexao)

# <font color='red' style='font-size: 40px;'> Funções </font>
<hr style='border: 2px solid red;'>

# <font color='red' style='font-size: 40px;'> 1.0 Entendimento das Bases de Dados </font>
<hr style='border: 2px solid red;'>

## 1.1) DEMOGRAPHICS

> 1. Customer ID: Um ID exclusivo que identifica cada cliente.

> 2. Count: Um valor usado em relatórios para resumir o número de clientes em um conjunto filtrado.

> 3. Gender: Sexo do cliente: Masculino, Feminino.

> 4. Age: A idade atual do cliente, em anos, no momento em que o trimestre fiscal terminou.

> 5. Under 30: Indica se o cliente tem menos de 30 anos: Sim, Não.

> 6. Senior Citizen: Indica se o cliente tem 65 anos ou mais: Sim, Não. 

> 7. Married: Indica se o cliente é casado: Sim, Não.

> 8. Dependents: Indica se o cliente vive com um dependente: Sim, Não. Os dependentes podem ser filhos, pais, avós, etc.

> 9. Number of Dependents: Indica o número de dependentes que moram com o cliente.

## 1.2) LOCATIONS

> 1. Customer ID: Um ID exclusivo que identifica cada cliente.

> 2. Count: Um valor usado em relatórios para resumir o número de clientes em um conjunto filtrado.

> 3. Country: O país de residência principal do cliente.

> 4. State: O estado da residência principal do cliente.

> 5. City: A principal cidade de residência do cliente.

> 6. Zip Code: O código postal da residência principal do cliente.

> 7. Latitude: A latitude da residência principal do cliente.

> 8. Longitude: A longitude da residência principal do cliente.

## 1.3) POPULATIONS

> 1. ID: Um ID exclusivo que identifica cada linha.

> 2. Zip Code: O código postal da residência principal do cliente.

> 3. Population: Uma estimativa da população atual para toda a área do código postal.

## 1.4) SERVICES

> 1. Customer ID: Um ID exclusivo que identifica cada cliente.

> 2. Count: Um valor usado em relatórios para resumir o número de clientes em um conjunto filtrado.

> 3. Qaurter: O trimestre fiscal do qual os dados foram derivados (por exemplo, Q3).

> 4. Referred a Friend: Indica se o cliente já encaminhou um amigo ou familiar para esta empresa: Sim, Não

> 5. Number of Referrals: Indica o número de referências/indicações até esta data que o cliente realizou.

> 6. Tenure in Months: Indica o número total de meses em que o cliente está na empresa no final do trimestre especificado anteriormente.

> 7. Offer: Identifica a última oferta de marketing que o cliente aceitou, se aplicável. Os valores incluem Nenhum, Oferta

> 8. Phone Service: Indica se o cliente contrata o serviço de telefonia residencial com a empresa: Sim, Não

> 9. Avg Monthly Long Distance Charges: Indica as tarifas médias de longa distância do cliente, calculadas no final do trimestre especificado anteriormente.

> 10. Multiple Lines: Indica se o cliente contrata várias linhas telefônicas com a empresa: Sim, Não

> 11. Internet Service: ndica se o cliente contrata o serviço de Internet com a empresa: Não, Sim

> 12. Internet Type: O tipo de internet que o cliente usa DSL, Fibra Óptica, Cabo.

> 13. Avg Monthly GB Download: Indica o volume médio de downloads do cliente em gigabytes, calculado no final do trimestre especificado

> 14. Online Security: Indica se o cliente assina um serviço de segurança online adicional fornecido pela empresa: Sim, Não

> 15. Online Backup: Indica se o cliente assina um serviço de backup online adicional fornecido pela empresa: Sim, Não

> 16. Device protection Plan: Indica se o cliente assina um plano de proteção de dispositivo adicional para o equipamento de Internet fornecido 

> 17. Premium Tech Support: Indica se o cliente assina um plano de suporte técnico adicional da empresa com tempos de espera reduzidos

> 18. Streaming TV: Indica se o cliente usa seu serviço de Internet para transmitir programação de televisão de um provedor terceirizado

> 19. Streaming Movies: Indica se o cliente usa seu serviço de Internet para transmitir filmes de um provedor externo: Sim, Não. 

> 20. Streaming Music: Indica se o cliente usa seu serviço de Internet para transmitir música de um provedor externo: Sim, Não. A empresa não

> 21. Unlimited Data: Indica se o cliente pagou uma taxa mensal adicional para ter downloads/uploads de dados ilimitados

> 22. Contract: Indica o tipo de contrato atual do cliente: mês a mês, um ano, dois anos.

> 23. Paperless Billing: Indica se o cliente optou pelo faturamento sem papel: Sim, Não

> 24. Payment Method: Indica como o cliente paga sua fatura: Saque bancário, cartão de crédito, cheque enviado

> 25. Monthly Charge: Indica a cobrança mensal total atual do cliente para todos os serviços da empresa.

> 26. Total Charges: Indica os encargos totais do cliente, calculados no final do trimestre especificado anteriormente.

> 27. Total Refunds: Indica o total de reembolsos do cliente, calculado no final do trimestre especificado anteriormente.

> 28. Total Extra Data Charges: Indica os encargos totais do cliente para downloads de dados adicionais acima dos especificados em seu plano

> 29. Total Long Distance Charges: Indica o total de tarifas de longa distância do cliente acima das especificadas em seu plano, ao final do trimestre especificado

> 30. Total Revenue: Receita total

## 1.5) STATUS

> 1. Customer ID: Um ID exclusivo que identifica cada cliente.

> 2. Count: Um valor usado em relatórios para resumir o número de clientes em um conjunto filtrado.

> 3. Quarter: O trimestre fiscal do qual os dados foram derivados (por exemplo, Q3).

> 4. Customer Status: Indica o status do cliente no final do trimestre: Churned, Stayed ou Joined

> 5. Churn Label: Sim = o cliente deixou a empresa neste trimestre. Não = o cliente permaneceu na empresa. Diretamente relacionado

> 6. Churn Value: 1 = o cliente deixou a empresa neste trimestre. 0 = o cliente permaneceu na empresa. Diretamente relacionado ao Churn

> 7. Churn Category: Uma categoria de alto nível para o motivo da rotatividade de clientes: Atitude, Concorrente, Insatisfação, Outros,

> 8. Churn Reason: Motivo específico de um cliente para deixar a empresa. Diretamente relacionado à categoria Churn.

# <font color='red' style='font-size: 40px;'> 2.0 Análise Exploratória </font>
<hr style='border: 2px solid red;'>

### 2.1) Uma visão geral da população de clientes TELCO visando entender as características dos clientes TELCO. 
- Qual é a divisão por sexo? 
- Eles têm dependentes? 
- E as Idades? 
- E as Cidades? 
- etc.

In [ ]:
query = ''' 
SELECT 
SUM(CASE WHEN GENDER = 'M' THEN 1 ELSE 0 END) AS qt_masculino, 
SUM(CASE WHEN GENDER = 'F' THEN 1 ELSE 0 END) AS qt_feminino 
FROM DEMOGRAPHICS
''' 

distribuicao_genero = pd.read_sql(query, conexao)
distribuicao_genero

### 2.2) Uma exploração de perdas para entender melhor as suposições do Diretor sobre contratos mensais e a região de San Diego como a mais afetada. Algumas segmentações para entender a perda de clientes que saíram durante a reunião são:

- Grupo 1: Contrato mes-a-mes e idade > 64
- Grupo 2: Contrato mes-a-mes , idade < 64 e número de referências < 2
- Grupo 3: Contrato diferente de mes-a-mes, idade > 64 e número de referências < 2
- Grupo 4: Contrato diferente a mes-a-mes e antiguidade em meses < 40

    

### 2.3) Identificação de clientes que são mais propensos a deixarem a empresa e, destes clientes, os que possuem alta representatividade monetária para a TELCO:

- Clientes mais propensos a sair (não saíram, mas estariam em risco de sair)

- Dos Clientes mais propensos, classificar os de de alto valor monetário e mandar para a CEO

# <font color='red' style='font-size: 40px;'> 3.0 Pré-Processamento </font>
<hr style='border: 2px solid red;'>

# <font color='red' style='font-size: 40px;'> 4.0 Modelagem I </font>
<hr style='border: 2px solid red;'>

# <font color='red' style='font-size: 40px;'> 5.0 Modelagem II </font>
<hr style='border: 2px solid red;'>

# <font color='red' style='font-size: 40px;'> 6.0 Consolidação dos Resultados </font>
<hr style='border: 2px solid red;'>